In [52]:
import pandas as pd
from textblob import TextBlob

# Load datasets
news_df = pd.read_csv('../Data/raw_analyst_ratings.csv')
stock_df = pd.read_csv('../Data/AMZN_historical_data.csv')

# Convert date columns to datetime format, handle errors
news_df['date'] = pd.to_datetime(news_df['date'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce', format='%Y-%m-%d')

# Drop rows with NaT in the Date column
stock_df = stock_df.dropna(subset=['Date'])

# Convert Date column to the same format as news_df
stock_df['date'] = stock_df['Date'].dt.date
news_df['date'] = news_df['date'].dt.date

# Define new date range
start_date = pd.to_datetime('2011-01-01').date()
end_date = pd.to_datetime('2020-01-01').date()

# Filter datasets to the specified date range
news_df = news_df[(news_df['date'] >= start_date) & (news_df['date'] <= end_date)]
stock_df = stock_df[(stock_df['date'] >= start_date) & (stock_df['date'] <= end_date)]


In [53]:
# Calculate daily returns for stock data
stock_df['daily_return'] = stock_df['Close'].pct_change()

# Aggregate sentiment scores by date
news_df['sentiment'] = news_df['headline'].apply(lambda text: TextBlob(text).sentiment.polarity)
news_daily_sentiment = news_df.groupby('date')['sentiment'].mean().reset_index()

# Merge datasets by date including the 'Close' column
merged_df = pd.merge(stock_df[['date', 'daily_return', 'Close']], news_daily_sentiment, on='date', how='inner')

# Display the results
print("Merged DataFrame:")
print(merged_df.head())
print(merged_df.info())


Merged DataFrame:
         date  daily_return   Close  sentiment
0  2011-01-03           NaN  9.2110   0.023555
1  2011-01-04      0.004288  9.2505   0.026955
2  2011-01-05      0.013026  9.3710   0.032222
3  2011-01-06     -0.008324  9.2930   0.025913
4  2011-01-07     -0.001991  9.2745   0.042296
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2264 entries, 0 to 2263
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          2264 non-null   object 
 1   daily_return  2263 non-null   float64
 2   Close         2264 non-null   float64
 3   sentiment     2264 non-null   float64
dtypes: float64(3), object(1)
memory usage: 70.9+ KB
None


In [54]:

# Perform correlation analysis
if not merged_df.empty:
    # Calculate overall correlation between daily return and sentiment
    overall_return_sentiment_corr = merged_df[['daily_return', 'sentiment']].corr().iloc[0, 1]
    print(f'Overall correlation between stock returns and sentiment: {overall_return_sentiment_corr:.2f}')
    
    # Calculate correlation between Close price and sentiment
    overall_close_sentiment_corr = merged_df[['Close', 'sentiment']].corr().iloc[0, 1]
    print(f'Overall correlation between Close price and sentiment: {overall_close_sentiment_corr:.2f}')
else:
    print("Merged DataFrame is empty. No correlation to compute.")

Overall correlation between stock returns and sentiment: 0.04
Overall correlation between Close price and sentiment: -0.14
